## Data Scrape Reviews,Rating and Prodcut From Amazon ##

In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup 

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get("https://www.amazon.com/s?k=smart+phone&rh=n%3A2335752011%2Cp_n_feature_forty-one_browse-bin%3A119653281011&dc&ds=v1%3AHtywl1TSoxMCpykVDga%2FqZhVQcLD3ijV0Z8LuCEdvN0&crid=3OHE6MR44SP1H&qid=1734355462&rnid=119653280011&sprefix=%2Caps%2C284&ref=sr_nr_p_n_feature_forty-one_browse-bin_1")

In [4]:
html_data = BeautifulSoup(driver.page_source,'html.parser')

### Getting No of Pages

In [6]:
no_of_pages = int(html_data.find('span',{'class':'s-pagination-item s-pagination-disabled'}).text)

In [7]:
no_of_pages

135

### Getting ALl Review with Respect to Procducts

In [8]:
products = html_data.find_all("div",{'data-component-type':'s-search-result'})

In [ ]:
products

In [10]:
import time
from tqdm import tqdm

In [ ]:
base_url = "https://www.amazon.in"
# Dataset Data Columns
product_id = []
product_url = []
product_name = []
product_img = []
product_price = []
product_rating = []
product_review = []

user_name = []
review_url = []
review_rating = []
review_title = []
review_description = []


for i in tqdm(range(no_of_pages)):
    page_url ='https://www.amazon.in/s?k=smart+phone&rh=n%3A2335752011%2Cp_n_feature_forty-one_browse-bin%3A119653281011&dc&ds=v1%3AHtywl1TSoxMCpykVDga%2FqZhVQcLD3ijV0Z8LuCEdvN0&crid=3OHE6MR44SP1H&qid=1734355462&rnid=119653280011&sprefix=%2Caps%2C284&ref=sr_nr_p_n_feature_forty-one_browse-bin_1&page'+str(i+1)
    driver.get(page_url)
    html_data = BeautifulSoup(driver.page_source,'html.parser')  
    # Fetching Products from Each Pages    
    for product in products:

        # FOR PRODUCT
        # for Product Ids
        asin = product.get('data-asin')
        #  print(product) # fetch all the product tages
        name = product.find('h2',{'class':"a-size-medium a-spacing-none a-color-base a-text-normal"}).text
        # For image
        img = product.find('img')['src']
        # for Rating
        rating = product.find('span',{'class':'a-icon-alt'}).text.replace(" out of 5 stars","")

        # Calculating the price to decimal Error Handling
        price =""
        try:
            price ='$'+ product.find('span',{"class":"a-price-whole"}).text + product.find('span',{"class":"a-price-fraction"}).text
        except:
            price = product.find('span',{'class':'a-offscreen'})

        # For total Review
        total_no_of_reviews = product.find('span',{'class':"a-size-base s-underline-text"}).text
        #   for product url
        url =product.find('a',{'class':'a-link-normal s-line-clamp-2 s-link-style a-text-normal'}).get('href')
        product_review_page_url = base_url + url
        #  Redirect to Page Url to Extract the Details   
        driver.get(product_review_page_url)

        # Extracts the Review     
        html_data = BeautifulSoup(driver.page_source, 'html.parser')
        reviews = html_data.find_all('li',{'data-hook':"review"})

        for review in reviews:
            # FOR REVIEWS
            product_id.append(asin)
            product_url.append(base_url+url)
            product_name.append(name)
            product_img.append(img)
            product_rating.append(rating)
            product_price.append(price)
            product_review.append(total_no_of_reviews)

            #  for username 
            review_user_name = review.find('span',{'class':"a-profile-name"}).text
            user_name.append(review_user_name)

            # for review url including Error Handling
            product_review_url =""
            try:
                product_review_url =base_url+review.find('a',{'data-hook':"review-title"}).get('href')
            except:
                product_review_url=""
            review_url.append(product_review_url)

            # for review_title including Error Handling
            product_review_title =""
            try:
                product_review_title = review.find('a',{'data-hook':"review-title"}).text.replace(review.find('span',{"class":"a-icon-alt"}).text,"").replace('\n',"")
            except AttributeError:
                product_review_title = review.find('span',{'data-hook':"review-title"}).text.replace('\n',"")

            review_title.append(product_review_title)

            # for review_Rating
            product_review_rating = review.find('span',{"class":"a-icon-alt"}).text.replace(" out of 5 stars","")
            review_rating.append(product_review_rating)

            # for review_descritpion
            product_review_descritpion = review.find('div',{'data-hook':"review-collapsed"}).text.replace('\n',"")
            review_description.append(product_review_descritpion)
            

100%|██████████████████████████████████████████████████████████████████████████████| 135/135 [3:11:37<00:00, 85.17s/it]


In [ ]:
print(len(product_id),
      len(user_name),
      len(review_url),
      len(review_rating),
      len(review_title),
      len(review_description))

In [ ]:
data = pd.DataFrame({'product_id':product_id,
                     'product_overall_rating':product_rating,
                     'total_reviews':product_review,
                     'user_name':user_name,
                     'review_url':review_url,
                     'review_rating':review_rating,
                     'review_title':review_title,
                     "review_description":review_description
                    })

In [ ]:
data

In [ ]:
data.to_csv('data.csv')